# Notebook for parser development

In [1]:
## not for parser. for notebook only 

## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Loading data

Current approach: loading all files into 1 pandas dataframe (currently (4721, 21)). Then I can...
1. check the duplicates situation (key columns vs all columns) and raise errors if need be
2. remove duplicates before generating documents
3. Do some tasks column-wise over all the data, rather than while iterating over rows

If I did the generator approach (load files 1 by 1, 1 row at a time), I'd have to modify how I do things:
1. Don't do this check/raise errors. But try to mitigate potential "duplicate" issues: 
  * Sort all delimited strings
  * Use a hash of all column values (when they're all strings) for `_id`. Want rows with all the same values to produce the same hash
2. Either leave to BioThings toolset to remove duplicates, or could use a set of `_id` hashes so far to check/not create duplicate docs.
3. Do the tasks on single rows/chunks (pandas [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv) has an iterator for rows/chunks! see iterator/chunksize parameters)


Notes:
* There are a few existing parsers that use `pandas` to load the entire raw data file at once: https://github.com/search?q=repo%3Abiothings%2Fpending.api%20pandas&type=code
* But there are other existing parsers that use `csv` to load the file **one row at a time** (generator): https://github.com/search?q=repo%3Abiothings%2Fpending.api+csv+reader&type=code

In [2]:
## put into parser: already done

## python 3.12.9
import pathlib       ## 1.0.1 
import pandas as pd  ## 2.2.3 

## don't put in parser. Just for this notebook
import glob
from pprint import pprint

## unsure on putting into parser: more for notebook viewing/debugging...
pd.options.display.max_columns = None

In [3]:
## put into parser (format): DONE

base_file_path = pathlib.Path.home().joinpath("Desktop", "EBIgene2pheno_files", "From_FTP")

## pathlib's Path.glob produces a generator, vs glob.glob produces an array from cwd
glob.glob("*.csv.gz")
base_file_path.glob("*.csv.gz")

## using list works to check if paths matching pattern were actually found or not
all_file_paths = list(base_file_path.glob("*.csv.gz"))
all_file_paths

[]

<generator object Path.glob at 0x1118d68a0>

[PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/CardiacG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/SkeletalG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/DDG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/SkinG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/Hearing_lossG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/CancerG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/EyeG2P_2025-02-28.csv.gz')]

In [4]:
## put into parser (format): DONE


## ingest all columns as str
df = pd.concat((pd.read_csv(f, dtype=str) for f in all_file_paths), ignore_index=True)

## make column names snake-case - usable with itertuples later
df.columns = df.columns.str.replace(" ", "_")

df["date_of_last_review"].info(memory_usage="deep")
print("\n")
## change this column to datetime, saves memory
df["date_of_last_review"] = pd.to_datetime(df["date_of_last_review"])
df["date_of_last_review"].info(memory_usage="deep")

<class 'pandas.core.series.Series'>
RangeIndex: 4714 entries, 0 to 4713
Series name: date_of_last_review
Non-Null Count  Dtype 
--------------  ----- 
4714 non-null   object
dtypes: object(1)
memory usage: 377.6 KB


<class 'pandas.core.series.Series'>
RangeIndex: 4714 entries, 0 to 4713
Series name: date_of_last_review
Non-Null Count  Dtype              
--------------  -----              
4714 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 37.0 KB


In [ ]:
## worked before (pandas 2.0.3 and python 3.11.4) to ingest column as datetime
## now it doesn't: ingests as object/str instead
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=["date of last review"]) 
#                 for f in all_file_paths), ignore_index=True)

## the following also don't work
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=["date of last review"], 
#                            date_format="%Y-%m-%d %H:%M:%S%:z") 
#                 for f in all_file_paths), ignore_index=True)
## throws an error
# df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=[["date of last review"]], 
#                            date_format="%Y-%m-%d %H:%M:%S%:z") 
#                 for f in all_file_paths), ignore_index=True)
## throws an error
# df = pd.concat((pd.read_csv(f, dtype={"date of last review": pd.datetime64[ns, tz]})
#                 for f in all_file_paths), ignore_index=True)

In [5]:
df.shape
df.head()
df.info(memory_usage="deep")

(4714, 21)

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review
0,G2P00124,KCNE1,176261,6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,DD; Cardiac,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00
1,G2P00841,PTPN11,176876,9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,DD; Skin; Cardiac,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00
2,G2P03247,DSC2,125645,3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00
3,G2P03248,DSC2,125645,3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00
4,G2P03249,DSG2,125671,3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4714 entries, 0 to 4713
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              4714 non-null   object             
 1   gene_symbol                         4714 non-null   object             
 2   gene_mim                            4712 non-null   object             
 3   hgnc_id                             4714 non-null   object             
 4   previous_gene_symbols               4241 non-null   object             
 5   disease_name                        4714 non-null   object             
 6   disease_mim                         3574 non-null   object             
 7   disease_MONDO                       638 non-null    object             
 8   allelic_requirement                 4714 non-null   object             
 9   cross_cutting_modifier              629 n

## Checking, removing duplicates

In [6]:
## put into parser (format): DONE

## This is a data-quality check to make sure drop_duplicates actually removes all duplicates. 
## Based on exploring the data, the column subset below should uniquely define one 
##   record's data/row.
## If the de-deplicated data using this column set == de-duplicated data using the whole
##   dataset, then everything is fine and the parser can proceed with de-duplication.
## Else, the data needs to be explored and the parser probably needs adjustments.
## Many column values are delimited strings, and my concern is that these values could
##   differ (only in list order) for the "same data" in different files.

n_duplicates_column_combo = df[df.duplicated(subset=["g2p_id", "gene_symbol", "disease_name", "allelic_requirement", 
                                "molecular_mechanism"], keep=False)].shape

n_duplicates_all_columns = df[df.duplicated(keep=False)].shape

## for testing
# n_duplicates_all_columns = (1, 1)


if n_duplicates_column_combo != n_duplicates_all_columns: 
    raise AssertionError("The data format has changed, and record de-duplication may not work as-expected. " \
                          "Double-check the data and what columns uniquely define one record")

In [7]:
## put into parser (format): DONE

## drop duplicates
df.drop_duplicates(inplace=True, ignore_index=True)

In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3647 entries, 0 to 3646
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              3647 non-null   object             
 1   gene_symbol                         3647 non-null   object             
 2   gene_mim                            3645 non-null   object             
 3   hgnc_id                             3647 non-null   object             
 4   previous_gene_symbols               3277 non-null   object             
 5   disease_name                        3647 non-null   object             
 6   disease_mim                         2570 non-null   object             
 7   disease_MONDO                       561 non-null    object             
 8   allelic_requirement                 3647 non-null   object             
 9   cross_cutting_modifier              451 n

## Column-level transforms

In [9]:
df_diseasemim = df.copy()

## done to preserve NA
df_diseasemim["disease_mim"] = [i if pd.isna(i) \
                                else "OMIM:" + i if i.isnumeric() \
                                else i \
                                for i in df_diseasemim["disease_mim"]]

df_diseasemim["disease_mim"] = df_diseasemim["disease_mim"].str.replace("Orphanet", "orphanet")

In [10]:
df_diseasemim[df_diseasemim["disease_mim"].str.contains("OMIM:", na=False)].shape

df_diseasemim[df_diseasemim["disease_mim"].str.contains("orphanet:", na=False)].shape

## add up row count. If == num non-null in info above, you're good 
## right now 2570 == 2570, so good

(2561, 21)

(9, 21)

In [11]:
## put into parser (format): DONE

## COLUMN-LEVEL TRANSFORMS

## adding Translator/biolink prefixes to IDs
df["gene_mim"] = "OMIM:" + df["gene_mim"]
df["hgnc_id"] = "HGNC:" + df["hgnc_id"]
df["disease_mim"] = df["disease_mim"].str.replace("Orphanet", "orphanet")
## done to preserve NA
df["disease_mim"] = [i if pd.isna(i) \
                     else "OMIM:" + i if i.isnumeric() \
                     else i \
                     for i in df["disease_mim"]]

## strip whitespace
df["disease_name"] = df["disease_name"].str.strip()
df["comments"] = df["comments"].str.strip()

## create new columns
## UI really wants resource website urls like this. May need to adjust over time as website changes
df["g2p_record_url"] = "https://www.ebi.ac.uk/gene2phenotype/lgd/" +  df["g2p_id"]

## replace panel keywords with full names shown on G2P website for single record
## keeping "Hearing loss" as-is, changing all other values
df["panel"] = df["panel"].str.replace("DD", "Developmental disorders")
df["panel"] = df["panel"].str.replace("Cancer", "Cancer disorders")
df["panel"] = df["panel"].str.replace("Cardiac", "Cardiac disorders")
df["panel"] = df["panel"].str.replace("Eye", "Eye disorders")
df["panel"] = df["panel"].str.replace("Skeletal", "Skeletal disorders")
df["panel"] = df["panel"].str.replace("Skin", "Skin disorders")

In [12]:
## checking on column-level transforms

df.head()
# df["g2p record url"].unique()[0:100]

# df[df["disease mim"].str.contains("orphanet", na=False)]  ## 9 rows, so that's correct
# df[df["panel"].str.contains("Hearing", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
0,G2P00124,KCNE1,OMIM:176261,HGNC:6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,OMIM:612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,Developmental disorders; Cardiac disorders,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00124
1,G2P00841,PTPN11,OMIM:176876,HGNC:9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,OMIM:151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,Developmental disorders; Skin disorders; Cardi...,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00841
2,G2P03247,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03247
3,G2P03248,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03248
4,G2P03249,DSG2,OMIM:125671,HGNC:3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac disorders,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03249


## Generating documents

In [ ]:
for row in df.itertuples(index=False):
    if pd.notna(row.previous_gene_symbols):
        [i.strip() for i in row.previous_gene_symbols.split(";")]
        break

In [ ]:
## put into parser (format) -> DONE. 
##   don't save in array, yield each document instead

## GENERATING DOCS, saving in array
documents = []

## using itertuples because it's faster, preserves datatypes
for row in df.itertuples(index=False):
    ## simple assignments
    document = {
        "_id": row.g2p_id,
        "subject": {
            "hgnc_symbol": row.gene_symbol,
            "hgnc": row.hgnc_id,
            "type": "Gene"
        },
        "association": {
            "g2p_record_id": row.g2p_id,
            "g2p_record_url": row.g2p_record_url,
            "allelic_requirement": row.allelic_requirement,
            "confidence": row.confidence,
            "molecular_mechanism": row.molecular_mechanism,
            "molecular_mechanism_categorisation": row.molecular_mechanism_categorisation,
            "g2p_panels": [i.strip() for i in row.panel.split(";")],
            "date_of_last_review": str(row.date_of_last_review)
        },
        "object": {
            "name": row.disease_name,
            "type": "Disease"
        }
    }    
    ## only create field if value is not NA
    ##   if value is NA, list comprehension with split won't work
    ## Gene
    if pd.notna(row.gene_mim):
        document["subject"]["omim"] = row.gene_mim
    if pd.notna(row.previous_gene_symbols):
        document["subject"]["previous_gene_symbols"] = \
        [i.strip() for i in row.previous_gene_symbols.split(";")]
        
    ## Association
    if pd.notna(row.cross_cutting_modifier):
        document["association"]["cross_cutting_modifiers"] = [i.strip() for i in row.cross_cutting_modifier.split(";")]
    if pd.notna(row.variant_consequence):
        document["association"]["variant_consequences"] = [i.strip() for i in row.variant_consequence.split(";")]
    if pd.notna(row.variant_types):
        document["association"]["variant_types"] = [i.strip() for i in row.variant_types.split(";")]
    ## uses diff delimiters, could do more parsing
    if pd.notna(row.molecular_mechanism_evidence):
        document["association"]["molecular_mechanism_evidence"] = [i.strip() for i in row.molecular_mechanism_evidence.split("&")]
    if pd.notna(row.phenotypes):
        document["association"]["phenotypes"] = [i.strip() for i in row.phenotypes.split(";")]
    if pd.notna(row.publications):
        document["association"]["pmids"] = [i.strip() for i in row.publications.split(";")]
    if pd.notna(row.comments):
        document["association"]["curator_comments"] = row.comments
     
    ## Disease
    ## disease_mim: create field depending on whether OMIM or orphanet    
    if pd.notna(row.disease_mim):
        if row.disease_mim.startswith("orphanet"):
            document["object"]["orphanet"] = row.disease_mim
        elif row.disease_mim.startswith("OMIM"):
            document["object"]["omim"] = row.disease_mim
    if pd.notna(row.disease_MONDO):
        document["object"]["mondo"] = row.disease_MONDO
    
    documents.append(document)

## Checking documents

In [ ]:
df[df["panel"].str.contains("Hearing", na=False)]

## look for single values
# df[~ df["publications"].str.contains(";", na=True)]


# df[df["publications"].isna()]
# df[df["cross_cutting_modifier"].notna()]

In [ ]:
pprint(documents[3642])

# documents[416]

## Parser notes

Fine to use raise/assert in parser (raise is technically better programming behavior: https://realpython.com/python-assert-statement/#understanding-common-pitfalls-of-assert)


My notes on parser:
* adding prefixes to gene/disease IDs is good for pre-NodeNorming steps
* keeping diff gene/disease ID namespaces as separate fields right now is good for current BTE/x-bte-annotation system


My notes on syntax:
* use `yield` when you want to "return" within a "for loop" (return only happen once, then exit for-loop/function execution)
  * that's what it's used in main execution, when you're iterating over csv rows to generate documents
* use `yield from {function}` to get the data from a generator (created by `yield` being used the function)

## Pre-NodeNorming

In [13]:
## put into parser (format): DONE

import requests

## from BioThings annotator code: for interoperability between diff Python versions
# try:
#     from itertools import batched  # new in Python 3.12
# except ImportError:
#     from itertools import islice

#     def batched(iterable, n):
#         # batched('ABCDEFG', 3) → ABC DEF G
#         if n < 1:
#             raise ValueError("n must be at least one")
#         iterator = iter(iterable)
#         while batch := tuple(islice(iterator, n)):
#             yield batch

## doing to test that this works
from itertools import islice

def batched(iterable, n):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError("n must be at least one")
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        yield batch

nodenorm_url = "https://nodenorm.ci.transltr.io/get_normalized_nodes"

In [14]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3647 entries, 0 to 3646
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              3647 non-null   object             
 1   gene_symbol                         3647 non-null   object             
 2   gene_mim                            3645 non-null   object             
 3   hgnc_id                             3647 non-null   object             
 4   previous_gene_symbols               3277 non-null   object             
 5   disease_name                        3647 non-null   object             
 6   disease_mim                         2570 non-null   object             
 7   disease_MONDO                       561 non-null    object             
 8   allelic_requirement                 3647 non-null   object             
 9   cross_cutting_modifier              451 n

In [15]:
df.head()

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
0,G2P00124,KCNE1,OMIM:176261,HGNC:6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,OMIM:612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,Developmental disorders; Cardiac disorders,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00124
1,G2P00841,PTPN11,OMIM:176876,HGNC:9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,OMIM:151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,Developmental disorders; Skin disorders; Cardi...,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00841
2,G2P03247,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03247
3,G2P03248,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03248
4,G2P03249,DSG2,OMIM:125671,HGNC:3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac disorders,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03249


### Genes

Method #2: build a mapping dict from unique values

#### HGNC IDs

In [16]:
## put into parser (format): DONE

gene_hgnc_curies = df["hgnc_id"].dropna().unique()
len(gene_hgnc_curies)

2991

In [17]:
## put into parser (format): DONE

gene_nodenorm_mapping = {}

## larger batches are quicker
for batch in batched(gene_hgnc_curies, 1000):
    ## returns tuples
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    temp = {
        k: {"primary_id": v["id"]["identifier"],
            "primary_label": v["id"]["label"]} 
        for k,v in response.items()
    }
    gene_nodenorm_mapping.update(temp)

In [18]:
len(gene_nodenorm_mapping)

2991

#### OMIM IDs

In [19]:
gene_omim_curies = df["gene_mim"].dropna().unique()
len(gene_omim_curies)

2989

In [20]:
## larger batches are quicker
for batch in batched(gene_omim_curies, 1000):
    ## returns tuples
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    temp = {
        k: {"primary_id": v["id"]["identifier"],
            "primary_label": v["id"]["label"]} 
        for k,v in response.items()
    }
    gene_nodenorm_mapping.update(temp)

In [21]:
len(gene_nodenorm_mapping)

5980

#### Comparing the two

In [22]:
## look for differences in primary ID found if row has both IDs
for row in df[["gene_mim", "hgnc_id"]].itertuples(index=False):
    if pd.notna(row.gene_mim) and pd.notna(row.hgnc_id):
        if gene_nodenorm_mapping[row.gene_mim]["primary_id"] != \
        gene_nodenorm_mapping[row.hgnc_id]["primary_id"]:
            print(row)

## nothing prints, so there are no mismatches

In [23]:
## look for differences in name between NodeNormed and original data

for row in df[["gene_symbol", "hgnc_id"]].itertuples(index=False):
    if row.gene_symbol != gene_nodenorm_mapping[row.hgnc_id]["primary_label"]:
        print(f"G2P name {row.gene_symbol}, ID {row.hgnc_id}")
        print(f'NodeNorm name {gene_nodenorm_mapping[row.hgnc_id]["primary_label"]}, ID {gene_nodenorm_mapping[row.hgnc_id]["primary_id"]}')
        print("\n")
        
## mismatched names
## NodeNorm is correct for CENPJ -> CPAP and CCDC103 -> DNAAF19
## something is odd for the others (mitochondrial genes). Slack message for NodeNorm

G2P name MT-TP, ID HGNC:7494
NodeNorm name TRNP, ID NCBIGene:4571


G2P name CENPJ, ID HGNC:17272
NodeNorm name CPAP, ID NCBIGene:55835


G2P name CCDC103, ID HGNC:32700
NodeNorm name DNAAF19, ID NCBIGene:388389


G2P name MT-TL1, ID HGNC:7490
NodeNorm name TRNL1, ID NCBIGene:4567


G2P name MT-ND1, ID HGNC:7455
NodeNorm name ND1, ID NCBIGene:4535


G2P name MT-ND4, ID HGNC:7459
NodeNorm name ND4, ID NCBIGene:4538


G2P name MT-ATP6, ID HGNC:7414
NodeNorm name ATP6, ID NCBIGene:4508


G2P name MT-ND5, ID HGNC:7461
NodeNorm name ND5, ID NCBIGene:4540


G2P name MT-ND6, ID HGNC:7462
NodeNorm name ND6, ID NCBIGene:4541




Current plans for genes:
* only use HGNC ID column:
  * no missing values
  * no entity/primary ID mismatches using it vs OMIM
  * if there were entity/primary ID mismatches, I was going to prefer this mapping over OMIM's 
* use mapping dict to create two new columns: gene_nodenorm_id, gene_nodenorm_name

In [24]:
## put into parser (format): DONE

df["gene_nodenorm_id"] = [gene_nodenorm_mapping[i]["primary_id"] for i in df["hgnc_id"]]
df["gene_nodenorm_label"] = [gene_nodenorm_mapping[i]["primary_label"] for i in df["hgnc_id"]]

In [25]:
## look at the gene ID/name info

gene_mapping_df = df[["gene_symbol", "gene_mim", "hgnc_id", "gene_nodenorm_id", "gene_nodenorm_label"]].copy()
gene_mapping_df[gene_mapping_df["gene_mim"].isna()]

,gene_symbol,gene_mim,hgnc_id,gene_nodenorm_id,gene_nodenorm_label
1208,ZNF599,NaN,HGNC:26408,NCBIGene:148103,ZNF599
3440,MFSD6L,NaN,HGNC:26656,NCBIGene:162387,MFSD6L


In [26]:
## all NodeNormed IDs are NCBIGene
df[~ df["gene_nodenorm_id"].str.contains("NCBIGene:")]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url,gene_nodenorm_id,gene_nodenorm_label


### Diseases

#### OMIM/orphanet

In [27]:
## put into parser (format): DONE

disease_omim_orpha_curies = df["disease_mim"].dropna().unique()
len(disease_omim_orpha_curies)

2401

In [28]:
## put into parser (format): DONE

disease_nodenorm_mapping = {}

## larger batches are quicker
for batch in batched(disease_omim_orpha_curies, 1000):
    ## returns tuples
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        try:
            ## if NodeNorm did not recognize ID, v will be None
            if v is not None:
                ## some IDs aren't Diseases, throw those mappings out 
                if v["type"][0] == "biolink:Disease":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        disease_nodenorm_mapping.update(temp)
                    else:
                        print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            print(f'Encountered an error processing the NodeNorm response.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')
## old code: dict comprehension
#         temp = {
#             k: {"primary_id": v["id"]["identifier"],
#                 "primary_label": v["id"]["label"]} 
#             for k,v in response.items()
#             if v is not None if v["type"][0] == "biolink:Disease"
#         }
#         disease_nodenorm_mapping.update(temp)
#     except:
#         print(k)
#         pprint(v)

OMIM:601884: NodeNorm didn't recognize this ID
OMIM:133700: NodeNorm didn't recognize this ID
OMIM:249163: NodeNorm didn't recognize this ID
OMIM:300153: NodeNorm found different category biolink:Gene. Not keeping this mapping.
OMIM:300171: NodeNorm found different category biolink:Gene. Not keeping this mapping.
OMIM:319029: NodeNorm didn't recognize this ID
OMIM:613180: NodeNorm didn't recognize this ID
OMIM:188400: NodeNorm found different category biolink:Gene. Not keeping this mapping.
OMIM:300706: NodeNorm didn't recognize this ID
OMIM:611579: NodeNorm found different category biolink:Gene. Not keeping this mapping.
OMIM:237145: NodeNorm didn't recognize this ID
OMIM:300141: NodeNorm didn't recognize this ID
OMIM:319490: NodeNorm didn't recognize this ID
OMIM:603707: NodeNorm found different category biolink:Gene. Not keeping this mapping.
OMIM:601789: NodeNorm found different category biolink:Gene. Not keeping this mapping.
OMIM:300630: NodeNorm didn't recognize this ID
OMIM:603

Reviewing the output above: issues with NodeNorm process

In [29]:
df[df["disease_mim"] == "OMIM:608890"]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url,gene_nodenorm_id,gene_nodenorm_label
2951,G2P02516,SNAI2,OMIM:602150,HGNC:11094,SLUG; SLUGH; SLUGH1; SNAIL2,SNAI2-related Waardenburg syndrome,OMIM:608890,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,HP:0000598; HP:0008527; HP:0000152; HP:0001100...,12444107,Skin disorders,NaN,2018-07-05 15:56:51+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02516,NCBIGene:6591,SNAI2


In [30]:
len(disease_nodenorm_mapping)

print(f"Didn't resolve {len(disease_omim_orpha_curies) - len(disease_nodenorm_mapping)}")

2333

Didn't resolve 68


I checked all 26 cases where NodeNorm category was something else (was always Gene instead): 
* 1 case (OMIM:188400) where NodeNorm seemed to be wrong - and it is a disease -> Slack message to NodeNorm
* all other cases: NodeNorm is correct, this is a gene


I checked a few cases where NodeNorm didn't resolve (39 total): 
* 1 case (OMIM:601884) where the category was confusing -> Slack message to NodeNorm
* 1 case (OMIM:133700) that NodeNorm ought to have ingested as a disease -> Slack message to NodeNorm
* 1 case (OMIM:249163) where ID actually didn't exist
* 2 cases (OMIM:607236, OMIM:608890) where the ID was replaced/moved to a diff ID 


I checked all 3 cases where NodeNorm didn't find primary label:
* 2 cases (OMIM:620987, OMIM:620964) - good ID, NodeNorm is just missing the label -> Slack message to NodeNorm
* OMIM:300129 doesn't seem to be a disease. Not sure it should even be a entity/be in NodeNorm -> Slack message to NodeNorm


I checked all 9 orphanet IDs: mappings look fine

In [31]:
df[df["disease_mim"].str.contains("orphanet", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url,gene_nodenorm_id,gene_nodenorm_label
2052,G2P02555,SMAD6,OMIM:602931,HGNC:6772,HST17432; MADH6; MADH7,SMAD6-related non-syndromic craniosynostosis,orphanet:139390,NaN,monoallelic_autosomal,NaN,limited,absent gene product,NaN,loss of function,inferred,NaN,HP:0001363,27606499; 28808027,Developmental disorders,NaN,2019-04-17 12:18:34+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02555,NCBIGene:4091,SMAD6
2061,G2P02564,TONSL,OMIM:604546,HGNC:7801,IKBR; NFKBIL2,TONSL-related sponastrime dysplasia,orphanet:93357,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,HP:0005281; HP:0100255; HP:0002650; HP:0004322,30773277; 30773278,Developmental disorders,NaN,2018-11-07 09:53:40+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02564,NCBIGene:4796,TONSL
2846,G2P01819,RET,OMIM:164761,HGNC:9967,CDHF12; CDHR16; HSCR1; MEN2A; MEN2B; MTC1; PTC...,RET-related medullary thyroid carcinoma,orphanet:1332,NaN,monoallelic_autosomal,NaN,definitive,uncertain,NaN,undetermined,inferred,NaN,HP:0002865,10323403; 11454140; 14602786; 15240641; 950672...,Skin disorders; Cancer disorders,NaN,2017-09-01 16:19:16+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P01819,NCBIGene:5979,RET
2967,G2P02669,INSR,OMIM:147670,HGNC:6091,CD220,INSR-related leprechaunism,orphanet:508,NaN,biallelic_autosomal,NaN,definitive,absent gene product,NaN,loss of function,inferred,NaN,HP:0000119; HP:0001574; HP:0000138; HP:0000212...,8105179; 7815442,Skin disorders,NaN,2019-09-16 15:47:00+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02669,NCBIGene:3643,INSR
3197,G2P03354,SEC23B,OMIM:610512,HGNC:10702,CDA-II; CDAII; CDAN2; HEMPAS,SEC23B-related Cowden syndrome,orphanet:201,NaN,monoallelic_autosomal,NaN,limited,uncertain,NaN,gain of function,inferred,NaN,HP:0012114; HP:0012056; HP:0500009; HP:0005584...,26522472,Cancer disorders,NaN,2022-11-30 08:49:50+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03354,NCBIGene:10483,SEC23B
3304,G2P02001,RCBTB1,OMIM:607867,HGNC:18243,CLLD7; CLLL7; FLJ10716,RCBTB1-related familial exudative vitreoretino...,orphanet:891,NaN,monoallelic_autosomal,NaN,limited,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0012231; HP:0000006; HP:0011532; HP:0001141...,26908610,Eye disorders,NaN,2017-06-11 18:14:48+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02001,NCBIGene:55213,RCBTB1
3484,G2P02222,REV3L,OMIM:602776,HGNC:9968,POLZ; REV3,REV3L-related Moebius syndrome,orphanet:570,NaN,monoallelic_autosomal,NaN,limited,uncertain,NaN,undetermined,inferred,NaN,NaN,26068067,Eye disorders,NaN,2017-08-30 12:00:00+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02222,NCBIGene:5980,REV3L
3526,G2P02286,TMEM98,OMIM:615949,HGNC:24529,DKFZP564K1964,TMEM98-related nanophthalmos,orphanet:35612,NaN,monoallelic_autosomal,NaN,strong,uncertain,NaN,undetermined,inferred,NaN,NaN,24852644; 26392740,Eye disorders,NaN,2017-08-31 13:26:51+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02286,NCBIGene:26022,TMEM98
3607,G2P02426,TENM3,OMIM:610083,HGNC:29944,KIAA1455; ODZ3; TEN-M3; TEN3,TENM3-related colobomatous microphthalmia,orphanet:98938,NaN,biallelic_autosomal,NaN,strong,absent gene product,NaN,loss of function,inferred,NaN,HP:0000567; HP:0007700; HP:0000565; HP:0000482...,29753094; 27103084; 22766609,Eye disorders,NaN,2018-05-25 09:59:48+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P02426,NCBIGene:55714,TENM3


In [32]:
disease_nodenorm_mapping["orphanet:1332"]

{'primary_id': 'MONDO:0015277',
 'primary_label': 'medullary thyroid gland carcinoma'}

In [35]:
## put into parser (format): DONE

## using this kind of logic to create columns for function

# [disease_nodenorm_mapping[i]["primary_label"] if disease_nodenorm_mapping.get(i) 
#  else pd.NA for i in df["disease_mim"]]

#### MONDO

In [ ]:
## put into parser (format): 

disease_mondo_curies = df["disease_MONDO"].dropna().unique()
len(disease_mondo_curies)

In [ ]:
## put into parser (format): 

## larger batches are quicker
for batch in batched(disease_mondo_curies, 1000):
    ## returns tuples
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        try:
            ## if NodeNorm did not resolve ID, v will be None
            if v is not None:
                ## some IDs aren't Diseases o_0, throw them out
                ## also throw out mapping if no primary label found
                if v["type"][0] == "biolink:Disease":
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        disease_nodenorm_mapping.update(temp)
                    else:
                        print(f"{k}: NodeNorm didn't find primary label")
                else:
                    print(f'{k}: NodeNorm category is {v["type"][0]}')
            else:
                print(f"{k}: NodeNorm didn't resolve")
        except:
            print(k)
            pprint(v)
## old code: dict comprehension
#         temp = {
#             k: {"primary_id": v["id"]["identifier"],
#                 "primary_label": v["id"]["label"]} 
#             for k,v in response.items()
#             if v is not None if v["type"][0] == "biolink:Disease"
#         }
#         disease_nodenorm_mapping.update(temp)
#     except:
#         print(k)
#         pprint(v)

In [ ]:
len(disease_nodenorm_mapping)

#### Comparing the two

In [ ]:
## look for differences in primary ID found if row has both IDs

for row in df[["disease_mim", "disease_MONDO"]].itertuples(index=False):
    if pd.notna(row.disease_mim) and pd.notna(row.disease_MONDO):
        if disease_nodenorm_mapping.get(row.disease_mim):
            if disease_nodenorm_mapping[row.disease_mim]["primary_id"] != \
            disease_nodenorm_mapping[row.disease_MONDO]["primary_id"]:
                print(row)

## so...there's some mismatches

In [ ]:
df[df["disease_mim"] == "OMIM:610019"]
df[df["disease_MONDO"] == "MONDO:0005129"]

In [ ]:
disease_nodenorm_mapping["OMIM:610019"]
# disease_nodenorm_mapping["OMIM:614583"]
disease_nodenorm_mapping["MONDO:0005129"]

#### Summary so far

If both IDs exist, using omim is better. 

However, I'm now a little concerned about their disease name -> ID mapping process. 

Larger picture: 2392 OMIM disease IDs (2401 - 9 orphanet) and only a little of that got flagged in my code (68 + 22 = 90. < 90/2392, aka <3.8%) 

But the errors that have happened are kinda concerning: 
* if the row has 2 disease IDs, 1 of them can be just plain incorrect
* problems from the 68 flagged/not-mapped OMIM IDs above (and I didn't review all of them so there may be more issues):
  * 25 OMIM IDs were actually genes
  * a few OMIM IDs were confusing (OMIM:601884 maybe a disease, OMIM:300129 doesn't seem to be a disease)
  * some IDs don't exist
  * some IDs are outdated -> replaced/moved to diff IDs

Current plan:
* use disease OMIM IDs
* use logic in omim section for printing issues with mapping, only saving some mappings
* only create nodenorm mapping if it existed. Saving input ID that was used for mapping.


In [ ]:
## look for differences in name between NodeNormed and original data

for row in df[["gene_symbol", "hgnc_id"]].itertuples(index=False):
    if row.gene_symbol != gene_nodenorm_mapping[row.hgnc_id]["primary_label"]:
        print(f"G2P name {row.gene_symbol}, ID {row.hgnc_id}")
        print(f'NodeNorm name {gene_nodenorm_mapping[row.hgnc_id]["primary_label"]}, ID {gene_nodenorm_mapping[row.hgnc_id]["primary_id"]}')
        print("\n")
        
## mismatched names
## NodeNorm is correct for CENPJ -> CPAP and CCDC103 -> DNAAF19
## something is odd for the others (mitochondrial genes). Slack message for NodeNorm

### older code chunks

#### Method 1 (row-wise, will send duplicates)

In [ ]:
node_normed_gene_id = []
node_normed_gene_name = []
tally = 0

for row in df.itertuples(index=False):
    ## collect non-NA gene IDs for a row
    gene_curies = [i for i in [row.gene_mim, row.hgnc_id] if pd.notna(i)]
    
    ## gene_curies aren't empty
    if gene_curies:
        req_body = {
            "curies": gene_curies,
            "conflate": True
        }
        
        r = requests.post(nodenorm_url, json=req_body)
        response = r.json()
        
        if len(gene_curies) == 2:
            if response[gene_curies[0]]["id"]["identifier"] != \
            response[gene_curies[1]]["id"]["identifier"]:
                print(gene_curies)
            ## take HGNC ID always
            node_normed_gene_id.append(response[gene_curies[1]]["id"]["identifier"])
            node_normed_gene_name.append(response[gene_curies[1]]["id"]["label"])
        else:
            ## HGNC ID should be first
            node_normed_gene_id.append(response[gene_curies[0]]["id"]["identifier"])
            node_normed_gene_name.append(response[gene_curies[0]]["id"]["label"])
    
    tally += 1
    if tally % 10 == 0:
        print(tally)

In [ ]:
## collect non-NA gene IDs for a row

row_gene_curies = [i for i in [df.loc[0, "gene_mim"], df.loc[0, "hgnc_id"]] if pd.notna(i)]

In [ ]:
row_gene_curies

In [ ]:
## aka not empty
if row_curies:
    parameters = {
        ## use gene curies from 1 row
        "curie": row_gene_curies,
        "conflate": True
    }
    r = requests.get(nodenorm_url, params=parameters)
    response = r.json()

In [ ]:
response.keys()

In [ ]:
if response[row_gene_curies[0]]["id"]["identifier"] != \
   response[row_gene_curies[1]]["id"]["identifier"]:
    print(row_gene_curies)

In [ ]:
## NodeNormed primary/canonical ID and name
response["OMIM:176261"]["id"]
response["HGNC:6240"]["id"]

In [ ]:
if response[row_omim]:
    print("yay")

In [ ]:
## doesn't include NA by default
df["gene_symbol"].nunique()
df["gene_mim"].nunique()
df["hgnc_id"].nunique()

## ...so some repetitiveness to doing things row-wise. 

Figuring out logic: what's the best way to query NodeNorm -> get responses and handle?

Doing 1-by-1 is too slow. 





Will be sending duplicate queries in both cases

idea: do row-wise. This is the way the parser works anyways, especially for large datasets
- for genes: could use only hgnc (all rows have this). But want to check if there's any discrepancies with omim ID first. 



other idea: try to send an entire column/array at a time
this is small enough that I can probably send the entire column/array at a time
but still...



